# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [27]:
# 1) Seu código aqui

url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"

response = requests.get(url)
data = response.json()

df = pd.DataFrame(data['items'])

df_rj_municipios = df[df['uf'] == 'RJ']

df_rj_municipios.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
212,32249,Município,Queimados,3304144,RJ,PVL02.001397/2019-55,Arquivado a pedido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,5.000000e+06,0,0,22/09/2023
427,60354,Município,Paraty,3303807,RJ,PVL02.006783/2022-39,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado do Rio de Janeiro,Real,4.794057e+06,0,1,28/07/2022
502,65517,Município,Nova Iguaçu,3303500,RJ,PVL02.000206/2023-14,Deferido,17944.102874/2023-48,2023-07-25T17:16:18Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado do Rio de Janeiro,Real,5.000000e+07,0,1,28/07/2023
611,62993,Município,Paraíba do Sul,3303708,RJ,PVL02.000025/2023-98,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,3.900000e+06,0,1,28/03/2023
812,9750,Estado,Rio de Janeiro,33,RJ,None,Deferido,17944.001046/2006-92,2006-07-07T03:00:00Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,3.057373e+08,1,0,13/07/2006


In [28]:
# 2) Seu código aqui

status_frequencies = df_rj_municipios['status'].value_counts()

top_three_status = status_frequencies.head(3)

top_three_status

status
Deferido                                                 31
Encaminhado à PGFN com manifestação técnica favorável    12
Arquivado                                                10
Name: count, dtype: int64

In [33]:
# 3) Seu código aqui

df_rj_municipios.loc[:, 'data_status'] = df_rj_municipios['data_status'].astype(str)

df_rj_municipios.loc[:, 'ano_status'] = df_rj_municipios['data_status'].str.split('/').str[-1]

df_rj_municipios.loc[:, 'ano_status'] = df_rj_municipios['ano_status'].astype(int)


print(df_rj_municipios[['data_status', 'ano_status']].head())


    data_status ano_status
212  22/09/2023       2023
427  28/07/2022       2022
502  28/07/2023       2023
611  28/03/2023       2023
812  13/07/2006       2006


In [34]:
# 4) Seu código aqui

year_frequencies = df_rj_municipios['ano_status'].value_counts()

year_frequencies

ano_status
2007    9
2023    7
2014    7
2022    6
2008    6
2013    5
2009    5
2011    4
2012    4
2015    3
2010    3
2017    3
2016    2
2006    1
2004    1
2003    1
2002    1
2021    1
2019    1
2018    1
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [42]:
#1) Seu código aqui

def consultar_pvl(uf, tipo_interessado):
    url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl'

    response = requests.get(url)
    data = response.json()

    df = pd.DataFrame(data['items'])

    if tipo_interessado == 'Estado':
        df_uf = df[df['uf'] == uf]
    elif tipo_interessado == 'Município':
        df_uf = df[df['uf'] == uf]


    return df_uf

df_minas_gerais = consultar_pvl('MG', 'Estado')

df_minas_gerais.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
11,29603,Município,Astolfo Dutra,3104601,MG,PVL02.002130/2018-02,Arquivado a pedido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Banco do Brasil S/A,Real,193780.0,0,0,29/05/2020
15,39051,Município,Campo Florido,3111408,MG,PVL02.007723/2019-38,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,500000.0,1,1,30/12/2019
25,33964,Município,Varginha,3170701,MG,PVL02.002973/2019-81,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,4041000.0,1,1,13/11/2019
33,42994,Município,Ouro Preto,3146107,MG,PVL02.001391/2020-11,Encaminhado à PGFN com manifestação técnica fa...,17944.103507/2020-19,2020-08-19T17:24:49Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,45000000.0,0,1,28/08/2020
35,36568,Município,Timóteo,3168705,MG,PVL02.005460/2019-22,Arquivado por decurso de prazo (PVL-IF),None,None,Operação contratual interna,"Aquisição de terrenos, imóveis e/ou construção...",Instituição Financeira Nacional,Banco de Desenvolvimento de Minas Gerais S/A,Real,3000000.0,0,0,10/03/2023


In [45]:
# 2) Seu código aqui

df_minas_gerais = consultar_pvl('MG', 'Estado')
count_arquivado = df_minas_gerais[df_minas_gerais['status'] == 'Arquivado por decurso de prazo'].shape[0]

print(f"Solicitações para o Estado de Minas Gerais com status 'Arquivado por decurso de prazo': {count_arquivado}")

Solicitações para o Estado de Minas Gerais com status 'Arquivado por decurso de prazo': 63


In [58]:
# 3) Seu código aqui

url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"

response = requests.get(url)
data = response.json()

df = pd.DataFrame(data['items'])

df_ba_municipios = df[df['uf'] == 'BA']

df_ba_municipios.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
32,42878,Município,Jacobina,2917508,BA,PVL02.001389/2020-42,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,3518025.27,0,1,01/09/2020
42,46461,Município,Dom Macedo Costa,2910206,BA,PVL02.001978/2021-10,Arquivado a pedido,17944.103242/2021-30,2021-10-27T20:48:02Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2500000.00,0,0,04/11/2021
51,44789,Município,Cristópolis,2909703,BA,PVL02.001040/2021-91,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,1400000.00,0,1,18/10/2021
105,46532,Município,Itaguaçu da Bahia,2915353,BA,PVL02.002606/2021-01,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,2600000.00,0,0,28/09/2021
117,37368,Município,Barreiras,2903201,BA,PVL02.005683/2019-90,Deferido,17944.103411/2019-17,2019-09-09T21:42:59Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,40000000.00,1,1,13/09/2019


In [62]:
# Consultar os dados para o estado da Bahia com tipo de interessado 'Município'
df_bahia_municipios = consultar_pvl('BA', 'Município')

# Filtrar apenas as solicitações deferidas
df_bahia_deferidas = df_bahia_municipios[df_bahia_municipios['status'] == 'Deferido']

# Verificar se há duplicatas e remover se houver
df_bahia_deferidas = df_bahia_deferidas.drop_duplicates(subset=['interessado'])

# Identificar o município com mais solicitações deferidas
municipio_max_deferidas = df_bahia_deferidas['interessado'].iloc[0]

print(f"Município da Bahia com mais solicitações deferidas: {municipio_max_deferidas}")


Município da Bahia com mais solicitações deferidas: Barreiras


In [63]:
# 4) Seu código aqui

df_bahia_estado = consultar_pvl('BA', 'Estado')

df_bahia_estado.to_csv('solicitacoes_bahia_estado.csv', index=False)
print("Dados salvos com sucesso em 'solicitacoes_bahia_estado.csv'.")

Dados salvos com sucesso em 'solicitacoes_bahia_estado.csv'.
